In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
#@ Loading the required libraries
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /home/kushal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
#@ Loading the required dataset
data_train = pd.read_csv('../data/train.csv')
data_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [23]:
#@ Checking shape of dataset
data_train.shape

(20800, 5)

In [24]:
#@ Checking if there is any missing values
data_train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [25]:
#@ Let's drop the null values
data = data_train.dropna()

In [26]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [27]:
#@ Let's get Independent and Dependent features
X = data.drop('label', axis=1)
y = data['label']

In [28]:
#@ Checking shape after extracting features
print('The shape of X is:',X.shape)
print('The shape of y is:',y.shape)

The shape of X is: (18285, 4)
The shape of y is: (18285,)


In [29]:
#@ Defining vocabulary size
voc_size=5000

**One Hot Representation**

In [30]:
#@ Copying the X values as messages
messages = X.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [31]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [32]:
messages.reset_index(inplace=True)
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [33]:
#@ Let's apply the preprocessing in the dataset
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-z0-9]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic [photos]',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay 2 5 million releas everyon arrest dakota access pipelin',
 'fbi c

In [34]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [35]:
data['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [36]:
#@ Now, let's apply OHE in the corpus
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[2223, 3980, 293, 4890, 3980, 2846, 1505, 1609, 2771, 2105],
 [3067, 1649, 1905, 485, 3777, 88, 4122],
 [4952, 4857, 4706, 2553],
 [1991, 3625, 659, 2595, 863, 4323, 2523],
 [851, 3777, 3865, 4195, 1083, 4877, 3777, 1294, 3989, 2756],
 [1965,
  3284,
  988,
  2206,
  2720,
  4316,
  2374,
  716,
  2363,
  1264,
  4702,
  4238,
  107,
  2396,
  4122],
 [1196, 1056, 115, 3176, 4568, 406, 1531, 4113, 3312, 305, 4782],
 [2659, 4250, 1889, 3388, 4882, 4298, 4316, 4432, 3312, 305, 4782],
 [4888, 415, 816, 3668, 3256, 4442, 573, 4016, 4316, 966],
 [4087, 4659, 2483, 1212, 3243, 4646, 359, 188],
 [4363, 4452, 2429, 2912, 346, 1302, 571, 4859, 4181, 3620, 993],
 [863, 257, 3980, 4442, 4316, 4882],
 [2186, 3381, 235, 3159, 4898, 1977, 1734, 467, 1914],
 [451, 3065, 4605, 4109, 4296, 3496, 1331, 3312, 305, 4782],
 [2273, 4166, 3885, 1831, 2956, 3312, 305, 4782],
 [4876, 4627, 3527, 4257, 2191, 138, 599, 2260, 4738, 2371, 2206, 265],
 [1201, 2999, 1649],
 [608, 2097, 3900, 2595, 4316, 1522, 3338,

In [37]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [38]:
onehot_repr[0]

[2223, 3980, 293, 4890, 3980, 2846, 1505, 1609, 2771, 2105]

**Embedding representation**

In [39]:
#@ Let's apply post padding in each sentences
sent_length=20
embedded_docs = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)
embedded_docs

array([[2223, 3980,  293, ...,    0,    0,    0],
       [3067, 1649, 1905, ...,    0,    0,    0],
       [4952, 4857, 4706, ...,    0,    0,    0],
       ...,
       [ 563, 2244,  710, ...,    0,    0,    0],
       [ 547, 4882,  179, ...,    0,    0,    0],
       [3540, 2575, 1442, ...,    0,    0,    0]], dtype=int32)

In [40]:
embedded_docs[0]

array([2223, 3980,  293, 4890, 3980, 2846, 1505, 1609, 2771, 2105,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [41]:
#@ Now, let's create the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100,))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

2023-04-13 17:26:32.766762: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


2023-04-13 17:26:33.557769: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-13 17:26:33.560433: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-13 17:26:33.562079: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [42]:
len(embedded_docs), y.shape

(18285, (18285,))

In [43]:
X_final = np.array(embedded_docs)
y_final=np.array(y)

In [44]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [45]:
#@ Applying train/test/splt
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
#@ Let's finally train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10


2023-04-13 17:26:34.343068: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-13 17:26:34.345881: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-13 17:26:34.347714: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

190/192 [============================>.] - ETA: 0s - loss: 0.2963 - accuracy: 0.8558

2023-04-13 17:26:40.933546: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-13 17:26:40.935571: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-13 17:26:40.937241: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

192/192 [==============================] - 8s 25ms/step - loss: 0.2951 - accuracy: 0.8564 - val_loss: 0.1953 - val_accuracy: 0.9171
Epoch 2/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1303 - accuracy: 0.9516 - val_loss: 0.2082 - val_accuracy: 0.9220
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0845 - accuracy: 0.9705 - val_loss: 0.2260 - val_accuracy: 0.9201
Epoch 4/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0552 - accuracy: 0.9820 - val_loss: 0.2554 - val_accuracy: 0.9165
Epoch 5/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0393 - accuracy: 0.9872 - val_loss: 0.3325 - val_accuracy: 0.9114
Epoch 6/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0255 - accuracy: 0.9922 - val_loss: 0.3184 - val_accuracy: 0.9133
Epoch 7/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0247 - accuracy: 0.9923 - val_loss: 0.3791 - val_accuracy: 0.8974
Epoch 8/10

- The model seems to perform well on the training data, with the training accuracy improving from 0.84 in the first epoch to 0.9961 in the ninth epoch. However, the validation accuracy does not improve as much, hovering around 0.91 throughout training.
- This suggests that the model is overfitting to the training data and not generalizing well to new data. The increasing validation loss in later epochs also supports this hypothesis. The model may benefit from additional regularization techniques, such as dropout or weight decay, to prevent overfitting.

In [46]:
#@ Adding Dropout layer to the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100,))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


2023-04-13 17:27:19.661773: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-13 17:27:19.664132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-13 17:27:19.666083: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [47]:
#@ Applying train/test/split
X_final = np.array(embedded_docs)
y_final=np.array(y)
#@ Applying train/test/splt
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
#@ Let's finally train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10


2023-04-13 17:27:20.126205: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-13 17:27:20.128911: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-13 17:27:20.131232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

191/192 [============================>.] - ETA: 0s - loss: 0.3215 - accuracy: 0.8400

2023-04-13 17:27:26.155518: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-13 17:27:26.157654: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-13 17:27:26.159216: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

192/192 [==============================] - 7s 25ms/step - loss: 0.3212 - accuracy: 0.8401 - val_loss: 0.1927 - val_accuracy: 0.9191
Epoch 2/10
192/192 [==============================] - 4s 22ms/step - loss: 0.1390 - accuracy: 0.9478 - val_loss: 0.1982 - val_accuracy: 0.9215
Epoch 3/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0946 - accuracy: 0.9656 - val_loss: 0.1980 - val_accuracy: 0.9246
Epoch 4/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0672 - accuracy: 0.9771 - val_loss: 0.2367 - val_accuracy: 0.9152
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0522 - accuracy: 0.9825 - val_loss: 0.2725 - val_accuracy: 0.9180
Epoch 6/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0401 - accuracy: 0.9866 - val_loss: 0.2927 - val_accuracy: 0.9170
Epoch 7/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0329 - accuracy: 0.9891 - val_loss: 0.3110 - val_accuracy: 0.9097
Epoch 8/10

- The result after applying dropout technique shows a slightly better validation accuracy than the previous result. This means that dropout helped reduce overfitting and improve the generalization of the model.
- The training accuracy decreased slightly, which is expected because dropout randomly drops out some connections during training, making it harder for the model to fit the training data perfectly. However, this trade-off is necessary to achieve better performance on unseen data.

In [48]:
# Define early stopping criteria
earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')

# Adding Dropout layer to the model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100,))
model.add(Dense(1, activation='sigmoid'))

2023-04-13 17:28:09.561729: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-13 17:28:09.564204: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-13 17:28:09.566317: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [49]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model with early stopping
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, callbacks=[earlystop])

Epoch 1/100


2023-04-13 17:28:10.068344: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-13 17:28:10.071164: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-13 17:28:10.073590: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

192/192 [==============================] - ETA: 0s - loss: 0.3263 - accuracy: 0.8416

2023-04-13 17:28:17.050910: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-13 17:28:17.052959: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-13 17:28:17.054543: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

192/192 [==============================] - 8s 29ms/step - loss: 0.3263 - accuracy: 0.8416 - val_loss: 0.1982 - val_accuracy: 0.9165
Epoch 2/100
192/192 [==============================] - 5s 24ms/step - loss: 0.1432 - accuracy: 0.9439 - val_loss: 0.2029 - val_accuracy: 0.9216
Epoch 3/100
192/192 [==============================] - 5s 24ms/step - loss: 0.1013 - accuracy: 0.9635 - val_loss: 0.2076 - val_accuracy: 0.9165
Epoch 4/100
192/192 [==============================] - 5s 25ms/step - loss: 0.0740 - accuracy: 0.9739 - val_loss: 0.2365 - val_accuracy: 0.9205
Epoch 4: early stopping


- The use of callbacks has stopped training early on the 6th epoch since the validation loss did not improve in the last 3 epochs. This is a good sign that our model is not overfitting and has reached a good level of accuracy. The final validation accuracy achieved by the model is 91.10%.

In [50]:
#@ Performance Metrics and Accuracy
y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.6, 1, 0)

 12/189 [>.............................] - ETA: 0s  

2023-04-13 17:28:32.930383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-13 17:28:32.932820: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-13 17:28:32.934793: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

189/189 [==============================] - 1s 4ms/step


In [51]:
#@ Confusion Matrix
confusion_matrix(y_test, y_pred)

array([[3148,  271],
       [ 231, 2385]])

In [52]:
#@ Calculating accuracy score
print('The accuracy score is:',accuracy_score(y_test, y_pred))

The accuracy score is: 0.9168185584092792


In [53]:
#@ Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      3419
           1       0.90      0.91      0.90      2616

    accuracy                           0.92      6035
   macro avg       0.91      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035



**The END**